In [1]:
import os
import cv2
import torch
import shutil
import numpy as np
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.cluster import KMeans
from torchvision.io import read_image
from torchvision.transforms.functional import to_pil_image

In [2]:
# Paths
INPUT_FOLDER = r"C:\Users\nguye\OneDrive\Desktop\2024\Spring 24\ResreachWithGollum\input_images"  # Folder containing input images
FEATURE_FOLDER = r"C:\Users\nguye\OneDrive\Desktop\2024\Spring 24\ResreachWithGollum\features"  # Output folder for clustered segments

# Ensure output directory exists
if os.path.exists(FEATURE_FOLDER):
    shutil.rmtree(FEATURE_FOLDER)  # Clear previous results
os.makedirs(FEATURE_FOLDER)


In [3]:
# Load Pretrained CNN (Inception-V3)
device = "cuda" if torch.cuda.is_available() else "cpu"
weights = models.Inception_V3_Weights.DEFAULT
model = models.inception_v3(weights=weights).to(device)
model.eval()

# Image Transform for Feature Extraction
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Resize for Inception-V3
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
def extract_features(image):
    """Extract features from an image segment using Inception-V3."""
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        features = model(image).cpu().numpy().flatten()  # Feature vector
    return features

def segment_image(image_path):
    """Segment the image using OpenCV and return bounding boxes and cropped regions."""
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Use Canny edge detection
    edges = cv2.Canny(gray, 100, 200)
    
    # Find contours from edges
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    segments = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w > 20 and h > 20:  # Ignore very small segments
            segment = image[y:y+h, x:x+w]
            segments.append((segment, (x, y, w, h)))  # Store image segment and its bounding box
    
    return segments


In [7]:
# Step 1: Process all images and extract segments
all_segments = []
segment_images = []

for filename in os.listdir(INPUT_FOLDER):
    if filename.lower().endswith((".jpg", ".png", ".jpeg")):
        img_path = os.path.join(INPUT_FOLDER, filename)
        segments = segment_image(img_path)
        
        for segment, bbox in segments:
            segment_images.append(segment)
            pil_image = to_pil_image(cv2.cvtColor(segment, cv2.COLOR_BGR2RGB))  # Convert OpenCV image to PIL
            features = extract_features(pil_image)
            all_segments.append(features)

In [8]:
# Step 2: Cluster segments using K-Means
NUM_CLUSTERS = 5  # Adjust based on expected number of feature groups (e.g., wheels, windows)
all_segments = np.array(all_segments)

kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42, n_init=10)
labels = kmeans.fit_predict(all_segments)

In [9]:
# Step 3: Save segments into corresponding feature folders
for idx, (segment, label) in enumerate(zip(segment_images, labels)):
    feature_dir = os.path.join(FEATURE_FOLDER, f"feature{label+1}")
    os.makedirs(feature_dir, exist_ok=True)

    # Save segmented image
    segment_path = os.path.join(feature_dir, f"segment_{idx}.jpg")
    cv2.imwrite(segment_path, segment)

print(f"Segmentation and clustering complete! Results saved in '{FEATURE_FOLDER}/'.")

Segmentation and clustering complete! Results saved in 'C:\Users\nguye\OneDrive\Desktop\2024\Spring 24\ResreachWithGollum\features/'.
